# セッション6 – マルチステップパイプライン (計画 → 実行 → 改良)

ルーティング関数を使用して各ステップごとにモデルを選択し、最終的な答えを改良します。


### 説明: 依存関係のインストール
ステップごとのチャット呼び出しに必要な `foundry-local-sdk` と `openai` をインストールします。再実行しても問題ありません。


# シナリオ
マルチステップモデルパイプライン: (1) タスクを個別のステップに計画する、(2) 意図に基づいたモデル選択で各ステップを実行する、(3) 最終的な統合された回答を洗練する。効率を保ちながら品質を維持するために異種SLM機能を連鎖させることを示します。


In [13]:
!pip install -q foundry-local-sdk openai

### 説明: コアインポート
意図検出のための正規表現、エイリアスごとの添付ファイル管理のためのFoundry Local manager、そしてチャット完了のためのOpenAIクライアントをインポートします。


In [14]:
import re
from foundry_local import FoundryLocalManager
from openai import OpenAI

### 説明: 機能カタログとルール
機能対応のカタログと、ステップテキストを意図カテゴリ（コード、要約、分類、一般）にマッピングするために使用される正規表現ルールを定義します。複数のモデルが意図をサポートしている場合、優先度が低い値が優先されます。


In [15]:
CATALOG = {
 'phi-4-mini': {'capabilities':['general','summarize'],'priority':2},
 'qwen2.5-coder-7b': {'capabilities':['code','refactor'],'priority':1},
 'qwen2.5-0.5b': {'capabilities':['classification','fast'],'priority':3},
}
RULES = [
 (re.compile('code|refactor|function', re.I), 'code'),
 (re.compile('summari|abstract|tl;dr', re.I), 'summarize'),
 (re.compile('classif|category|label', re.I), 'classification'),
]

### 説明: 意図、モデル選択 & チャットヘルパー
提供内容:
- `detect_intent`: 正規表現ベースの分類用。
- `pick_model`: 機能と優先度に基づいて最適なエイリアスを選択。
- `chat`: エイリアスごとにクライアントをインスタンス化し、単一ターンの応答を返す便利なラッパー。


In [16]:
def detect_intent(text: str):
    """Return an intent label based on simple regex rules; falls back to 'general'."""
    for pat, intent in RULES:
        if pat.search(text):
            return intent
    return 'general'

def pick_model(intent: str) -> str:
    """Pick the best model for an intent using capability match first, then priority."""
    ranked = []
    for name, meta in CATALOG.items():
        ranked.append((name, intent in meta['capabilities'], meta['priority']))
    # Sort: capability match (True first), then lower priority value
    ranked.sort(key=lambda t: (not t[1], t[2]))
    return ranked[0][0]

def chat(alias: str, content: str, temp: float = 0.4) -> str:
    """Simple helper to send a single user message to a Foundry Local model via OpenAI client."""
    m = FoundryLocalManager(alias)
    client = OpenAI(base_url=m.endpoint, api_key=m.api_key or 'not-needed')
    mid = m.get_model_info(alias).id
    resp = client.chat.completions.create(
        model=mid,
        messages=[{'role': 'user', 'content': content}],
        max_tokens=180,
        temperature=temp,
    )
    return resp.choices[0].message.content

### 説明: マルチステップパイプライン関数
パイプラインを実装します: 計画 → ステップを解析 → 意図に基づくルーティングで各ステップを実行 → 結合された出力を洗練。検査や評価のための構造化された辞書を返します。


In [17]:
def pipeline(task: str):
    """Multi-step pipeline: plan → execute steps → refine final answer.

    Returns dict with keys: plan (raw plan text), steps (list of tuples), final (refined answer).
    Each step tuple: (index, step_text, step_result, model_alias_used)
    """
    # 1. Plan
    plan_alias = pick_model('general')
    plan_prompt = (
        "Break the task into 3 concise, actionable steps (no extra commentary).\n"
        f"Task: {task}"
    )
    plan = chat(plan_alias, plan_prompt)

    # 2. Parse steps (robust to numbering or bullet styles)
    raw_lines = [l.strip() for l in plan.splitlines() if l.strip()]
    steps = []
    for line in raw_lines:
        cleaned = re.sub(r'^\d+[).:-]?\s*', '', line)  # remove leading numbering
        cleaned = re.sub(r'^[-*]\s*', '', cleaned)      # remove bullet markers
        if cleaned:
            steps.append(cleaned)
        if len(steps) == 3:
            break
    if not steps:
        steps = [task]

    # 3. Execute steps
    outputs = []
    for idx, step in enumerate(steps, 1):
        intent = detect_intent(step)
        exec_alias = pick_model(intent)
        exec_prompt = (
            f"Execute step {idx} for the overall task.\n"
            f"Overall task: {task}\n"
            f"Step {idx}: {step}\n"
            "Return a concise result focusing only on the step objective."
        )
        result = chat(exec_alias, exec_prompt)
        outputs.append((idx, step, result, exec_alias))

    # 4. Refine final answer
    refine_alias = pick_model('summarize')
    combined = "\n\n".join(
        f"Step {idx} ({alias}) Output:\n{res}" for idx, _step, res, alias in outputs
    )
    refine_prompt = (
        "You are a senior assistant. Synthesize these step outputs into a cohesive final answer.\n"
        "Ensure clarity, avoid repetition, and highlight improvements or key insights if relevant.\n\n"
        f"Task: {task}\n\n"
        f"Step Outputs:\n{combined}"
    )
    final_ans = chat(refine_alias, refine_prompt)

    return {"plan": plan, "steps": outputs, "final": final_ans}

### 説明: 実行例タスク
リファクタリングを目的としたタスクで、コードと要約といった混合の意図を示す完全なパイプラインを実行します。結果の辞書には、未加工の計画、各ステップの出力（選択されたモデルのエイリアス付き）、そして最終的に合成された回答が含まれています。


In [18]:
result = pipeline('Generate a refactored version of a slow Python loop and summarize performance gains.')
result

{'plan': '1. Profile the existing slow Python loop to identify bottlenecks.\n2. Refactor the loop using optimized techniques (e.g., list comprehensions, map, or itertools).\n3. Compare the performance of the refactored loop with the original using a benchmarking tool and summarize the gains.',
 'steps': [(1,
   'Profile the existing slow Python loop to identify bottlenecks.',
   "To execute step 1, you would use a profiling tool like `cProfile` in Python to analyze the performance of the existing slow loop. Here's an example of how you might do this:\n\n```python\nimport cProfile\n\ndef slow_loop():\n    # Example of a slow loop\n    result = []\n    for i in range(1000000):\n        result.append(i * i)\n    return result\n\n# Profile the slow loop\ncProfile.run('slow_loop()', 'profile_stats')\n\n# To see the results, you can use pstats module\nimport pstats\np = pstats.Stats('profile_stats')\np.sort_stats('cumulative').print_stats()\n```\n\nThis code will run the `slow_loop` function

### 説明: 結果オブジェクトの表示
迅速な確認や後続の評価（例: ステップの品質や改良の有効性を測定するため）を目的とした、構造化されたパイプライン出力を表示します。



---

**免責事項**:  
この文書は、AI翻訳サービス[Co-op Translator](https://github.com/Azure/co-op-translator)を使用して翻訳されています。正確性を追求しておりますが、自動翻訳には誤りや不正確な部分が含まれる可能性があります。元の言語で記載された文書を正式な情報源としてお考えください。重要な情報については、専門の人間による翻訳を推奨します。この翻訳の使用に起因する誤解や誤解釈について、当方は一切の責任を負いません。
